In [2]:
import cv2
import os
import glob
import numpy as np

import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedShuffleSplit
from keras.applications import MobileNetV2
from keras.layers import Dense,GlobalAveragePooling2D

In [3]:
batch_size        = 32
num_classes       = 2
save_dir          = os.path.join(os.getcwd(), 'saved_models')
model_name        = 'dogscats.h5'

In [4]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(160,160),
        batch_size=32,
        class_mode='binary')

# validation_generator = val_datagen.flow_from_directory(
#         'validation',
#         target_size=(160, 160),
#         batch_size=32,
#         class_mode='binary')


Found 2000 images belonging to 2 classes.


In [5]:
mobilenet = MobileNetV2(input_shape = (160,160,3), weights='imagenet',include_top=False)

In [6]:
mobilenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [7]:
x=mobilenet.output
x=GlobalAveragePooling2D()(x)
preds=Dense(1,activation='sigmoid')(x) #final layer with softmax activation
model=Model(inputs=mobilenet.input,outputs=preds)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [9]:
for layer in model.layers:
#     print(layer.name)
    layer.trainable=False

print(model.layers[-1].name)
model.layers[-1].trainable = True
    

dense_1


In [10]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
epochs = 50
# step_size_train= train_generator.n//train_generator.batch_size
# step_size_val = validation_generator.n//validation_generator.batch_size

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,                    
                    epochs=10)
                    

# model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, 
#                     validation_data = validation_generator , validation_steps = step_size_val, 
#                     epochs=epochs)

Epoch 1/10


/home/alexander/anaconda3/envs/tf/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/alexander/anaconda3/envs/tf/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


62/62 [==============================] - 229s 4s/step - loss: 0.4405 - acc: 0.7893
Epoch 2/10
62/62 [==============================] - 140s 2s/step - loss: 0.2556 - acc: 0.8977
Epoch 3/10
62/62 [==============================] - 232s 4s/step - loss: 0.2114 - acc: 0.9158
Epoch 4/10
62/62 [==============================] - 222s 4s/step - loss: 0.1897 - acc: 0.9224
Epoch 5/10
62/62 [==============================] - 121s 2s/step - loss: 0.1678 - acc: 0.9405
Epoch 6/10
62/62 [==============================] - 73s 1s/step - loss: 0.1636 - acc: 0.9360
Epoch 7/10
62/62 [==============================] - 63s 1s/step - loss: 0.1563 - acc: 0.9350
Epoch 8/10
62/62 [==============================] - 62s 1s/step - loss: 0.1605 - acc: 0.9350
Epoch 9/10
62/62 [==============================] - 61s 980ms/step - loss: 0.1514 - acc: 0.9375
Epoch 10/10
62/62 [==============================] - 176s 3s/step - loss: 0.1370 - acc: 0.9446


In [12]:
model.save(model_name)